In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
from IPython.display import clear_output
import time
import catboost
import re
import optuna
import warnings
warnings.filterwarnings("ignore")
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, StackingClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (FunctionTransformer, StandardScaler, MinMaxScaler, RobustScaler, QuantileTransformer, PowerTransformer,
                                   OneHotEncoder)
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, KFold, StratifiedShuffleSplit
from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
data = pd.read_csv(r"D:\Intellipaat\kegal datasets\steel plates dataset\train.csv")
data

,id,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,...,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,0,584,590,909972,909977,16,8,5,2274,113,...,-0.5000,-0.0104,0.1417,0,0,0,1,0,0,0
1,1,808,816,728350,728372,433,20,54,44478,70,...,0.7419,-0.2997,0.9491,0,0,0,0,0,0,1
2,2,39,192,2212076,2212144,11388,705,420,1311391,29,...,-0.0105,-0.0944,1.0000,0,0,1,0,0,0,0
3,3,781,789,3353146,3353173,210,16,29,3202,114,...,0.6667,-0.0402,0.4025,0,0,1,0,0,0,0
4,4,1540,1560,618457,618502,521,72,67,48231,82,...,0.9158,-0.2455,0.9998,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19214,19214,749,757,143210,143219,17,4,4,2193,122,...,-0.1429,0.0044,0.2901,0,0,0,1,0,0,0
19215,19215,723,735,2488529,2488541,231,17,26,27135,104,...,0.7222,-0.0989,0.5378,0,0,0,0,0,0,1
19216,19216,6,31,1578055,1578129,780,114,98,71112,41,...,0.7719,-0.4283,0.9997,1,0,0,0,0,0,0
19217,19217,9,18,1713172,1713184,126,13,26,14808,88,...,0.9610,-0.1162,0.3509,0,0,0,0,0,0,1


In [3]:
col_name = []
with open(r"D:\Intellipaat\kegal datasets\steel plates dataset\Faults27x7_var") as f:
    for line in f:
        col_name.append(line.strip())

In [4]:
data  = data.drop(columns=['id'])

In [5]:
org = pd.read_csv("D:\Intellipaat\kegal datasets\steel plates dataset\Faults.NNA",engine= "python",delimiter= '\s',names= col_name)
org

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,...,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,42,50,270900,270944,267,17,44,24220,76,108,...,0.8182,-0.2913,0.5822,1,0,0,0,0,0,0
1,645,651,2538079,2538108,108,10,30,11397,84,123,...,0.7931,-0.1756,0.2984,1,0,0,0,0,0,0
2,829,835,1553913,1553931,71,8,19,7972,99,125,...,0.6667,-0.1228,0.2150,1,0,0,0,0,0,0
3,853,860,369370,369415,176,13,45,18996,99,126,...,0.8444,-0.1568,0.5212,1,0,0,0,0,0,0
4,1289,1306,498078,498335,2409,60,260,246930,37,126,...,0.9338,-0.1992,1.0000,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1936,249,277,325780,325796,273,54,22,35033,119,141,...,-0.4286,0.0026,0.7254,0,0,0,0,0,0,1
1937,144,175,340581,340598,287,44,24,34599,112,133,...,-0.4516,-0.0582,0.8173,0,0,0,0,0,0,1
1938,145,174,386779,386794,292,40,22,37572,120,140,...,-0.4828,0.0052,0.7079,0,0,0,0,0,0,1
1939,137,170,422497,422528,419,97,47,52715,117,140,...,-0.0606,-0.0171,0.9919,0,0,0,0,0,0,1


In [6]:
df = pd.concat([data,org],ignore_index= True)
df

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,...,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,584,590,909972,909977,16,8,5,2274,113,140,...,-0.5000,-0.0104,0.1417,0,0,0,1,0,0,0
1,808,816,728350,728372,433,20,54,44478,70,111,...,0.7419,-0.2997,0.9491,0,0,0,0,0,0,1
2,39,192,2212076,2212144,11388,705,420,1311391,29,141,...,-0.0105,-0.0944,1.0000,0,0,1,0,0,0,0
3,781,789,3353146,3353173,210,16,29,3202,114,134,...,0.6667,-0.0402,0.4025,0,0,1,0,0,0,0
4,1540,1560,618457,618502,521,72,67,48231,82,111,...,0.9158,-0.2455,0.9998,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21155,249,277,325780,325796,273,54,22,35033,119,141,...,-0.4286,0.0026,0.7254,0,0,0,0,0,0,1
21156,144,175,340581,340598,287,44,24,34599,112,133,...,-0.4516,-0.0582,0.8173,0,0,0,0,0,0,1
21157,145,174,386779,386794,292,40,22,37572,120,140,...,-0.4828,0.0052,0.7079,0,0,0,0,0,0,1
21158,137,170,422497,422528,419,97,47,52715,117,140,...,-0.0606,-0.0171,0.9919,0,0,0,0,0,0,1


In [7]:
X = df.iloc[:,:-7]

In [8]:
pastry = df['Pastry'].copy()
z_scratch = df['Z_Scratch'].copy()
k_scatch = df['K_Scatch'].copy()
stains = df['Stains'].copy()
dirtiness = df['Dirtiness'].copy()
bumps = df['Bumps'].copy()
other_faults = df['Other_Faults'].copy()

In [9]:
Y = [pastry, z_scratch, k_scatch, stains, dirtiness, bumps, other_faults]
Y_col_name = ['pastry', 'z_scratch', 'k_scatch', 'stains', 'dirtiness', 'bumps', 'other_faults']

In [10]:
y_params_dict = {}
for y_nm in Y_col_name:
    y_params_dict[y_nm] = None

In [11]:
y_scores = []

In [12]:
xgbc = XGBClassifier(min_child_samples = 15)

In [13]:
xgbc.fit(X ,pastry)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_samples=15, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

In [12]:
for y, y_name in zip(Y, Y_col_name):
    print(f'trial: {y_name}\n')

    def objective_xgb(model):
        cvo = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
        params = dict(
            n_estimators = model.suggest_int('n_estimators', 100, 500),
            max_depth = model.suggest_int('max_depth', 2, 64),
            max_leaves = model.suggest_int('num_leaves', 2, 128),
            learning_rate = model.suggest_float('learning_rate', 0.001, 0.3),
            min_child_samples = model.suggest_int('min_child_samples', 2, 500),
            min_child_weight = model.suggest_float('min_child_weight', 0.01, 10),
            subsample = model.suggest_float('subsample', 0.33, 0.85),
            colsample_bytree = model.suggest_float('colsample_bylevel', 0.33, 0.7),
            reg_alpha = model.suggest_float('reg_alpha', 0.001, 0.1),
            reg_lambda = model.suggest_float('reg_lambda', 0.001, 0.1),
            gamma = model.suggest_float('gamma',0.01 , 10)
        )
        XGB = XGBClassifier(random_state=0, objective='binary:logistic', verbose=-1, **params)

        pipe = Pipeline(
            steps=[
                ('scaler', StandardScaler()),
                ('model', XGB)
            ]
        )

        score = np.mean(cross_val_score(pipe, X, y, scoring='roc_auc', cv=cvo))
        return score

    study_xgb = optuna.create_study(direction='maximize')
    study_xgb.optimize(objective_xgb, n_trials=1000, n_jobs=-1, show_progress_bar=True)

    best_params = study_xgb.best_params
    y_params_dict[y_name] = best_params

    best_score = study_xgb.best_value
    y_scores.append((y_name, best_score))

    time.sleep(5)  
    clear_output() 
 

[I 2024-03-08 17:39:17,250] A new study created in memory with name: no-name-99cc8cc0-f197-47fb-82a8-d9760f3daed6


trial: bumps



  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
y_params_dict

NameError: name 'y_params_dict' is not defined

In [ ]:
y_scores

[('pastry', 0.8828398171249147)]